# Aplicaciones Prácticas de NLP con Transformers

## Guía de Ejercicios Progresivos

---

## Introducción

En este cuaderno vas a trabajar con aplicaciones reales de Procesamiento de Lenguaje Natural (NLP) utilizando modelos Transformer preentrenados. Cada ejercicio representa un caso de uso concreto que se implementa en empresas y organizaciones de Argentina y el mundo.

Los ejercicios están organizados de manera progresiva:

1. **Nivel Básico:** Moderación automática de comentarios en redes sociales
2. **Nivel Intermedio:** Extracción de información de currículums (RRHH)
3. **Nivel Avanzado:** Chatbot de soporte técnico con respuestas automáticas
4. **Desafío Autónomo:** Análisis integral de reseñas de restaurantes

### ¿Qué vas a aprender?

- A usar modelos preentrenados de Hugging Face para tareas reales
- A comprender cómo funcionan los pipelines de NLP en la práctica
- A modificar y experimentar con código para resolver problemas concretos
- A evaluar la calidad de las predicciones de los modelos
- A trabajar de manera autónoma en un problema real completo

### Requisitos previos

- Conocimientos básicos de Python
- Familiaridad con conceptos de Machine Learning
- Haber visto los cuadernos introductorios sobre clasificación de sentimientos

---

## Instalación de dependencias

Ejecutá la siguiente celda para instalar las librerías necesarias:

In [ ]:
# Instalación de librerías (ejecutar solo una vez)
!pip install transformers torch pandas -q

---

## Ejercicio 1: Moderación de Comentarios en Redes Sociales

### Contexto del problema

Imaginate que trabajás en el equipo de redes sociales de una empresa argentina de delivery de comida. Recibís cientos de comentarios diarios en Instagram y Facebook. Necesitás un sistema que identifique automáticamente los comentarios negativos o críticos para que el equipo de atención al cliente pueda responderlos con prioridad.

### Aplicación real

Este tipo de sistemas se usan en:
- Redes sociales de empresas (PedidosYa, Rappi, Mercado Libre)
- Plataformas de e-commerce
- Servicios de atención al cliente automatizada

### ¿Qué vamos a hacer?

1. Cargar un modelo de análisis de sentimientos en español
2. Analizar comentarios reales de clientes
3. Clasificarlos automáticamente (positivo, negativo, neutral)
4. Identificar cuáles requieren atención urgente

---

### Paso 1: Importar librerías y cargar el modelo

In [3]:
from transformers import pipeline
import pandas as pd

# Cargamos un modelo de análisis de sentimientos específico para español
# Este modelo fue entrenado con datos de redes sociales en español

try:
    clasificador = pipeline(
        "text-classification",
        model="finiteautomata/beto-sentiment-analysis"
    )
    print("✅ Modelo BETO cargado correctamente")
except Exception as e:
    print(f"❌ Error con el modelo BETO: {e}")
    print("🔄 Probando modelo alternativo RoBERTuito...")
    # Modelo alternativo si falla
    clasificador = pipeline(
        "text-classification",
        model="pysentimiento/robertuito-sentiment-analysis"
    )
    print("✅ Modelo alternativo BETO cargado correctamente")

Device set to use cuda:0


✅ Modelo BETO cargado correctamente


### Paso 2: Definir comentarios de ejemplo

Acá tenemos comentarios típicos que podría recibir una empresa de delivery:

In [4]:
comentarios = [
    "La comida llegó rapidísimo y estaba caliente. El repartidor re amable, excelente servicio!",
    "Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.",
    "Todo bien, llegó en tiempo y forma. Nada para destacar.",
    "Me encanta este servicio, siempre cumplen. Los re banco!",
    "Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total."
]

### Paso 3: Analizar los comentarios

In [5]:
# Procesamos todos los comentarios
resultados = clasificador(comentarios)

# Creamos un DataFrame para visualizar mejor los resultados
df_resultados = pd.DataFrame({
    'Comentario': comentarios,
    'Sentimiento': [r['label'] for r in resultados],
    'Confianza': [round(r['score'], 3) for r in resultados]
})

df_resultados

,Comentario,Sentimiento,Confianza
0,La comida llegó rapidísimo y estaba caliente. ...,POS,0.998
1,Tardaron DOS HORAS para traer una pizza fría. ...,NEG,0.999
2,"Todo bien, llegó en tiempo y forma. Nada para ...",POS,0.997
3,"Me encanta este servicio, siempre cumplen. Los...",POS,0.999
4,Se equivocaron con mi pedido OTRA VEZ. Ya es l...,NEG,0.999


### Paso 4: Identificar comentarios que requieren atención urgente

In [6]:
# Filtramos solo los comentarios negativos
comentarios_urgentes = df_resultados[df_resultados['Sentimiento'] == 'NEG']

print("COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:")
print("="*60)
for idx, fila in comentarios_urgentes.iterrows():
    print(f"\n[{idx+1}] {fila['Comentario']}")
    print(f"    Confianza: {fila['Confianza']*100:.1f}%")

COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:

[2] Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.
    Confianza: 99.9%

[5] Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total.
    Confianza: 99.9%


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Agregá 3 comentarios nuevos** a la lista (pueden ser inventados o reales)
2. **Ejecutá nuevamente** las celdas de análisis
3. **Respondé:**
   - ¿El modelo clasificó correctamente tus comentarios?
   No, clasificó mal 2 de 3 comentarios.
   Comentario 1 (positivo) : lo clasificó como NEGATIVO.
   Comentario 3 (irónico/negativo) : lo clasificó como POSITIVO

   - ¿Hubo algún comentario que esperabas que clasifique diferente?
   Esperaba que el primero fuera POSITIVO era un elogio, sin ironia y compuesto palabras como "recomiendo" y "riquisima". En cambio, el de ironia me esperaba un mal resultado.

   - ¿Qué pasa si escribís un comentario con ironía? (probá con algo como "Genial, 3 horas esperando, lo mejor!")
   Lo clasifica de manera incorrecta por lo que me hace concluir que efectivamente no interpretó la ironia. Entiende "excelente innovacion" como positivo en un contexto negativo, y con un porcentaje muy alto de confianza, 99%.

Escribí tus comentarios nuevos en la siguiente celda:

In [7]:
# TUS COMENTARIOS ACÁ
mis_comentarios = [
    "La hamburguesa estaba riquisima y llego caliente, como tiene que ser. Recomiendo.",
    "El pedido llegó correcto pero el envoltorio estaba un poco roto, nada grave",
    "Pedí sushi y llegó todo mezclado sin salsa ni palitos, todo tenia el mismo gusto, excelente innovacion!"
]

# Analizamos
mis_resultados = clasificador(mis_comentarios)

# Mostramos
df_mis_resultados = pd.DataFrame({
    'Comentario': mis_comentarios,
    'Sentimiento': [r['label'] for r in mis_resultados],
    'Confianza': [round(r['score'], 3) for r in mis_resultados]
})

df_mis_resultados

,Comentario,Sentimiento,Confianza
0,La hamburguesa estaba riquisima y llego calien...,NEG,0.624
1,El pedido llegó correcto pero el envoltorio es...,NEU,0.993
2,Pedí sushi y llegó todo mezclado sin salsa ni ...,POS,0.999


---

## Ejercicio 2: Extracción Automática de Información de CVs

### Contexto del problema

Trabajás en el área de Recursos Humanos de una consultora argentina. Recibís decenas de CVs por día y necesitás extraer rápidamente información clave: nombres de candidatos, empresas donde trabajaron, universidades, ciudades, y tecnologías que manejan.

### Aplicación real

Este tipo de sistemas se usan en:
- Plataformas de empleo (LinkedIn, Bumeran, Computrabajo)
- Sistemas de ATS (Applicant Tracking Systems)
- Automatización de procesos de RRHH

### ¿Qué vamos a hacer?

1. Cargar un modelo de Reconocimiento de Entidades Nombradas (NER)
2. Procesar un fragmento de CV
3. Extraer automáticamente: personas, organizaciones, ubicaciones
4. Organizar la información de manera estructurada

---

### Paso 1: Cargar el modelo de NER

In [8]:
# Cargamos un modelo de NER específico para español
# Este modelo fue entrenado para reconocer personas, organizaciones y lugares
extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens de la misma entidad
)

print("Modelo NER cargado correctamente.")

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Modelo NER cargado correctamente.


### Paso 2: Definir un fragmento de CV

Este es un ejemplo típico de cómo un candidato describe su experiencia:

In [12]:
cv_texto = """María Fernández es una desarrolladora full-stack con 5 años de experiencia.
Estudió Ingeniería en Sistemas en la Universidad de Buenos Aires (UBA) y se graduó en 2018.
Trabajó como desarrolladora backend en Mercado Libre durante 3 años, donde lideró la migración
de sistemas legacy a microservicios. Posteriormente se unió a Globant como tech lead,
coordinando equipos distribuidos entre Buenos Aires y Córdoba. Tiene experiencia en Python,
Java, Docker y Kubernetes. Actualmente reside en Palermo, Ciudad de Buenos Aires."""

print("CV a procesar:")
print(cv_texto)

CV a procesar:
MStefania Verónica Jimenez es Contadora Pública graduada en la Universidad de Buenos Aires (UBA) en 2020. 
Cuenta con más de 8 años de experiencia en el sector bancario, especializándose en gestión de riesgos, cumplimiento normativo 
y análisis financiero.  Se desempeño en el área de Business Risk Control Management en HSBC Bank Argentina, donde lidero procesos de evaluación y control de riesgos de proveedores, colaborando con las áreas de Finanzas, 
Legales y Tecnología. Anteriormente trabajó como analista senior de Customer Due Diligence, y previamente en atención al cliente 
en HSBC y American Express. Posee conocimientos en SQL, Python y Excel, además de estar en formación en Ciencia de Datos 
e Inteligencia Artificial. Se destaca por su perfil analítico, capacidad resolutiva y orientación a la mejora continua. 
Reside Palermo en la Ciudad Autónoma de Buenos Aires.


### Paso 3: Extraer entidades del CV

In [10]:
# Procesamos el texto del CV
entidades = extractor_ner(cv_texto)

# Creamos un DataFrame para visualizar mejor
df_entidades = pd.DataFrame(entidades)

# Mostramos solo las columnas relevantes
df_entidades[['entity_group', 'word', 'score']].round(3)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,word,score
0,PER,María Fernández,1.000
1,MISC,Ingeniería en Sistemas,0.998
2,ORG,Universidad de Buenos Aires,0.998
3,ORG,UBA,0.983
4,MISC,Mercado Libre,0.789
5,ORG,Glo,1.000
6,ORG,##bant,0.821
7,LOC,Buenos Aires,1.000
8,LOC,Córdoba,1.000
9,MISC,Py,0.999


### Paso 4: Organizar la información por categoría

Ahora vamos a separar las entidades por tipo para que sea más fácil revisarlas:

In [11]:
# Diccionario para organizar por tipo de entidad
info_extraida = {
    'PER': [],  # Personas
    'ORG': [],  # Organizaciones
    'LOC': []   # Ubicaciones
}

# Clasificamos cada entidad
for entidad in entidades:
    tipo = entidad['entity_group']
    palabra = entidad['word']
    if tipo in info_extraida:
        info_extraida[tipo].append(palabra)

# Mostramos la información organizada
print("INFORMACIÓN EXTRAÍDA DEL CV")
print("="*60)
print(f"\nCANDIDATO/A: {', '.join(info_extraida['PER'])}")
print(f"\nEMPRESAS/INSTITUCIONES: {', '.join(info_extraida['ORG'])}")
print(f"\nUBICACIONES: {', '.join(info_extraida['LOC'])}")

INFORMACIÓN EXTRAÍDA DEL CV

CANDIDATO/A: María Fernández

EMPRESAS/INSTITUCIONES: Universidad de Buenos Aires, UBA, Glo, ##bant

UBICACIONES: Buenos Aires, Córdoba, Palermo, Ciudad de Buenos Aires


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí un fragmento de CV ficticio** (o usá el tuyo si querés) con:
   - Nombre de la persona
   - Al menos 2 empresas o instituciones educativas
   - Al menos 2 ubicaciones (ciudades, barrios, países)

2. **Ejecutá el análisis** y verificá qué entidades detectó el modelo

3. **Respondé:**
   - ¿El modelo identificó correctamente todas las entidades?
   - ¿Hubo alguna entidad que no detectó? ¿Por qué creés que pasó?
   - ¿Detectó alguna entidad incorrectamente?

El modelo reconocio las entidades, en un principio pensé que no detectar a "Youtube" como organizacion era incorrecto pero en realidad el modelo está en lo correcto porque es una plataforma. Luego, le agregue a "Google" como la compañia dueña, y efectivamente lo detecto como organizacion.

Escribí tu CV ficticio en la siguiente celda:

In [21]:
# TU CV FICTICIO ACÁ
mi_cv = """Axel Jimenez es un creador de contenido argentino especializado en videojuegos.
Estudió la orientación en Administración en la Escuela Secundaria N°14 de Buenos Aires,
donde descubrió su interés por la gestión de proyectos y el trabajo en equipo.
Actualmente se dedica de forma profesional a su canal de YouTube propiedad de Google, donde produce videos y transmisiones en vivo
centrados en juegos como Roblox, GTA V, Minecraft y Fortnite.
Con más de 2 millones suscriptores, se destaca por su estilo cercano, su sentido del humor y su capacidad para crear comunidades activas.
Además, colabora con otros streamers y participa en eventos gaming en línea.
Reside en la Ciudad Autónoma de Buenos Aires."""

# Procesamos
mis_entidades = extractor_ner(mi_cv)

# Mostramos
df_mis_entidades = pd.DataFrame(mis_entidades)
print("Entidades detectadas:")
print(df_mis_entidades[['entity_group', 'word', 'score']].round(3))

# Organizamos
mi_info = {'PER': [], 'ORG': [], 'LOC': []}
for e in mis_entidades:
    if e['entity_group'] in mi_info:
        mi_info[e['entity_group']].append(e['word'])

print("\n" + "="*60)
print(f"CANDIDATO/A: {', '.join(mi_info['PER'])}")
print(f"EMPRESAS/INSTITUCIONES: {', '.join(mi_info['ORG'])}")
print(f"UBICACIONES: {', '.join(mi_info['LOC'])}")

Entidades detectadas:
   entity_group                             word  score
0           PER                     Axel Jimenez  0.855
1          MISC                   Administración  0.999
2           ORG          Escuela Secundaria N°14  0.994
3           LOC                     Buenos Aires  0.977
4          MISC                          YouTube  0.999
5           ORG                           Google  1.000
6          MISC                           Roblox  0.973
7          MISC                            GTA V  0.999
8          MISC                              Min  0.999
9          MISC                         ##ecraft  0.969
10         MISC                         Fortnite  0.994
11          LOC  Ciudad Autónoma de Buenos Aires  0.997

CANDIDATO/A: Axel Jimenez
EMPRESAS/INSTITUCIONES: Escuela Secundaria N°14, Google
UBICACIONES: Buenos Aires, Ciudad Autónoma de Buenos Aires


---

## Ejercicio 3: Chatbot de Soporte Técnico Automático

### Contexto del problema

Trabajás en una empresa que vende electrodomésticos online. Los clientes suelen hacer preguntas frecuentes sobre garantías, envíos y devoluciones. Querés automatizar las respuestas a estas consultas usando un sistema de Question Answering (QA) que pueda responder basándose en la información de tus políticas.

### Aplicación real

Este tipo de sistemas se usan en:
- Chatbots de atención al cliente (WhatsApp, web)
- Sistemas de FAQ automáticas
- Asistentes virtuales corporativos

### ¿Qué vamos a hacer?

1. Cargar un modelo de Question Answering en español
2. Definir un contexto (políticas de la empresa)
3. Hacer preguntas sobre ese contexto
4. Generar respuestas automáticas

---

### Paso 1: Cargar los modelos necesarios

In [23]:
try:
    # Intentamos con el modelo Question Answering
    qa_modelo = pipeline(
        "question-answering",
        model="PlanTL-GOB-ES/roberta-base-bne-sqac"
    )
    print("✅ Modelo QA cargado correctamente")
except Exception as e:
    print(f"❌ Error con el modelo QA: {e}")
    print("🔄 Probando modelo alternativo...")
    # Modelo alternativo más estable
    qa_modelo = pipeline(
        "question-answering",
        model="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
    )
    print("✅ Modelo alternativo QA cargado correctamente")

❌ Error con el modelo QA: PlanTL-GOB-ES/roberta-base-bne-sqac is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`
🔄 Probando modelo alternativo...


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Modelo alternativo QA cargado correctamente


### Paso 2: Definir el contexto (políticas de la empresa)

Este es el documento que el chatbot va a usar para responder preguntas:

In [32]:
politicas_empresa = """POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío de devolución. Una vez recibido y verificado
el producto, reintegramos el 100% del dinero en un plazo de 15 días hábiles.
"""

print("Contexto cargado:")
print(politicas_empresa)

Contexto cargado:
POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío

### Paso 3: Hacer preguntas al sistema

In [33]:
# Definimos preguntas típicas de clientes
preguntas = [
    "¿Cuánto dura la garantía?",
    "¿Cuánto tarda el envío a Capital Federal?",
    "¿Cuántos días tengo para devolver un producto?",
    "¿Qué necesito para hacer válida la garantía?",
    "¿El envío es gratis?"
]

# Procesamos cada pregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in preguntas:
    respuesta = qa_modelo(question=pregunta, context=politicas_empresa)
    print(f"\nPREGUNTA: {pregunta}")
    print(f"RESPUESTA: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

RESPUESTAS AUTOMÁTICAS DEL CHATBOT

PREGUNTA: ¿Cuánto dura la garantía?
RESPUESTA: 12 meses
Confianza: 11.2%
------------------------------------------------------------

PREGUNTA: ¿Cuánto tarda el envío a Capital Federal?
RESPUESTA: entre 24 y 48 horas hábiles
Confianza: 36.1%
------------------------------------------------------------

PREGUNTA: ¿Cuántos días tengo para devolver un producto?
RESPUESTA: 10 días
Confianza: 36.1%
------------------------------------------------------------

PREGUNTA: ¿Qué necesito para hacer válida la garantía?
RESPUESTA: presentar la factura de compra original
Confianza: 28.7%
------------------------------------------------------------

PREGUNTA: ¿El envío es gratis?
RESPUESTA: El envío es gratuito para compras superiores
a $50.000
Confianza: 26.5%
------------------------------------------------------------


### Paso 4: Crear una función interactiva de chatbot

Vamos a crear una función que simule un chatbot completo:

In [34]:
def chatbot_soporte(pregunta, contexto=politicas_empresa):
    """
    Función que simula un chatbot de soporte técnico.

    Args:
        pregunta (str): La pregunta del cliente
        contexto (str): El documento con las políticas de la empresa

    Returns:
        str: Respuesta formateada para el cliente
    """
    resultado = qa_modelo(question=pregunta, context=contexto)

    # Formateamos la respuesta de manera amigable
    confianza = resultado['score']

    if confianza > 0.5:
        respuesta = f"""Hola! Te respondo tu consulta:

{resultado['answer']}

¿Te fue útil esta información? Si necesitás más detalles, no dudes en consultarnos."""
    else:
        respuesta = """Hola! No encontré una respuesta clara a tu consulta en nuestras
políticas. Te recomiendo que te comuniques con nuestro equipo de atención al cliente
al 0800-XXX-XXXX para que puedan ayudarte mejor."""

    return respuesta

# Probamos la función
print(chatbot_soporte("¿Puedo devolver un producto después de 2 semanas?"))

Hola! No encontré una respuesta clara a tu consulta en nuestras
políticas. Te recomiendo que te comuniques con nuestro equipo de atención al cliente
al 0800-XXX-XXXX para que puedan ayudarte mejor.


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí 3 preguntas nuevas** que un cliente podría hacer sobre las políticas
2. **Probá el chatbot** con esas preguntas
3. **Modificá el contexto** (politicas_empresa) agregando información nueva, por ejemplo:
   - Formas de pago aceptadas
   - Horarios de atención
   - Información sobre instalación de productos
4. **Respondé:**
   - ¿Las respuestas fueron precisas?
   - ¿Hubo alguna pregunta que el modelo no pudo responder bien?
   - ¿Qué pasa si hacés una pregunta sobre algo que NO está en el contexto?

Usá las siguientes celdas para experimentar:

In [42]:
# Definimos preguntas típicas de clientes
mis_preguntas = [
    "¿Qué pasa si el producto viene con daños?",
    "¿Puedo cambiar un producto por otro?",
    "¿Hacen envíos los fines de semana?",
    "¿Que pasa si lo roban en el camino?",
    "¿Que pasa si por culpa del electrodomestico salta la termica?"
]

# Procesamos cada pregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in mis_preguntas:
    respuesta = qa_modelo(question=pregunta, context=politicas_empresa)
    print(f"\nPREGUNTA: {pregunta}")
    print(f"RESPUESTA: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

RESPUESTAS AUTOMÁTICAS DEL CHATBOT

PREGUNTA: ¿Qué pasa si el producto viene con daños?
RESPUESTA: El producto
debe estar sin usar
Confianza: 2.8%
------------------------------------------------------------

PREGUNTA: ¿Puedo cambiar un producto por otro?
RESPUESTA: El cliente
debe hacerse cargo del costo de envío de devolución
Confianza: 4.6%
------------------------------------------------------------

PREGUNTA: ¿Hacen envíos los fines de semana?
RESPUESTA: Realizamos envíos a todo el país
Confianza: 0.1%
------------------------------------------------------------

PREGUNTA: ¿Que pasa si lo roban en el camino?
RESPUESTA: El producto
debe estar sin usar
Confianza: 6.2%
------------------------------------------------------------

PREGUNTA: ¿Que pasa si por culpa del electrodomestico salta la termica?
RESPUESTA: daños físicos
Confianza: 0.4%
------------------------------------------------------------


In [48]:
# MODIFICÁ EL CONTEXTO ACÁ (agregá información nueva)
mi_contexto_ampliado = """POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

FORMAS DE PAGO: Aceptamos tarjetas de crédito (hasta 12 cuotas), débito, transferencia
bancaria y efectivo. Los pagos con tarjeta tienen un recargo del 5% para cubrir costos
de procesamiento. También aceptamos Mercado Pago y PayPal.

HORARIOS DE ATENCIÓN: Nuestro call center atiende de lunes a viernes de 8:00 a 20:00 hs
y sábados de 9:00 a 13:00 hs. Los envíos se realizan de lunes a viernes.

INSTALACIÓN: Ofrecemos servicio de instalación gratuita para productos mayores a $80.000.
Para productos menores, el costo de instalación es de $3.500. La instalación incluye
colocación y prueba de funcionamiento.

RETIRO EN LOCAL: Los clientes pueden retirar sus compras en nuestro local de
Av. Corrientes 1234, CABA. El horario de retiro es de lunes a viernes de 9:00 a 18:00 hs.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío de devolución. Una vez recibido y verificado
el producto, reintegramos el 100% del dinero en un plazo de 15 días hábiles.
"""

print("📝 CONTEXTO AMPLIADO CARGADO:")
print("Probando con información nueva...")

preguntas_nuevas = [
    "¿Cuáles son los horarios de atención?",
    "¿Ofrecen servicio de instalación?",
    "¿Aceptan tarjetas de crédito?",
    "¿Puedo retirar el producto en el local?",
    "¿Qué métodos de pago aceptan?",
    "¿Cuánto tarda el envío a Gran Buenos Aires?",
    "¿Cuánto dura la garantía?",
    "¿Puedo devolver un producto después de 2 semanas?",
    "¿Que pasa si le roban al repartidor?",
    "¿Puedo viajar a otro pais de vacaciones?",
    "¿Que pasa si me deja mi pareja?"
]

# Procesamos cada pregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in preguntas_nuevas:
    respuesta = qa_modelo(question=pregunta, context=mi_contexto_ampliado)
    print(f"\nPREGUNTA: {pregunta}")
    print(f"RESPUESTA: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

📝 CONTEXTO AMPLIADO CARGADO:
Probando con información nueva...
RESPUESTAS AUTOMÁTICAS DEL CHATBOT

PREGUNTA: ¿Cuáles son los horarios de atención?
RESPUESTA: 9:00 a 18:00 hs
Confianza: 13.6%
------------------------------------------------------------

PREGUNTA: ¿Ofrecen servicio de instalación?
RESPUESTA: instalación gratuita para productos mayores a $80.000
Confianza: 10.8%
------------------------------------------------------------

PREGUNTA: ¿Aceptan tarjetas de crédito?
RESPUESTA: Aceptamos tarjetas de crédito (hasta 12 cuotas)
Confianza: 11.3%
------------------------------------------------------------

PREGUNTA: ¿Puedo retirar el producto en el local?
RESPUESTA: Av. Corrientes 1234, CABA
Confianza: 7.3%
------------------------------------------------------------

PREGUNTA: ¿Qué métodos de pago aceptan?
RESPUESTA: tarjetas de crédito
Confianza: 7.2%
------------------------------------------------------------

PREGUNTA: ¿Cuánto tarda el envío a Gran Buenos Aires?
RESPUESTA: en

El contenido fue correcto pero incompleto en algunas preguntas como que metodos de pago aceptan que es general y en el texto hay varios metodos detallados, ademas nos muestra una baja confianza (7-14%) no reflejando la precisión del contenido. El modelo extrae fragmentos exactos del contexto, como un copy paste y no responde bien a síntesis de información.
Hice dos preguntas fuera de contexto, cuando le pregunte si puedo viajar a otro pais, me dijo "para el interior del pais" pero devolviendolo con baja confianza. Y el siguiente caso lo hizo con confianza 0%.

---

## Ejercicio 4: Desafío Autónomo - Análisis de Reseñas de Restaurantes

### Contexto del problema

Sos el encargado de marketing digital de una cadena de restaurantes porteña. Querés implementar un sistema inteligente que procese automáticamente las reseñas que los clientes dejan en Google Maps y redes sociales para:

1. Identificar si la reseña es positiva, negativa o neutral
2. Extraer información clave: nombres de platos mencionados, ubicaciones de las sucursales, nombres de empleados destacados
3. Responder automáticamente a preguntas frecuentes basándose en el menú y políticas del restaurante

### Aplicación real

Este tipo de sistemas combinados se usan en:
- Gestión de reputación online para cadenas de restaurantes
- Análisis de feedback de clientes en hotelería y turismo
- Sistemas de CRM (Customer Relationship Management) inteligentes
- Plataformas de delivery con análisis de satisfacción del cliente

---

### Tu tarea

**Este ejercicio lo tenés que resolver completamente solo**, aplicando todo lo que aprendiste en los ejercicios anteriores. No hay código de ejemplo, solo las instrucciones.

### Parte 1: Análisis de sentimiento de reseñas (30%)

1. Creá una lista con al menos 5 reseñas ficticias de clientes sobre un restaurante argentino (podés inventarlas o buscar reales)
2. Cargá un modelo de análisis de sentimientos en español (buscá en Hugging Face)
3. Clasificá cada reseña y mostrá los resultados en un DataFrame
4. Identificá cuántas reseñas son positivas, negativas y neutrales (si el modelo lo soporta)

**Pistas:**
- Usá `pipeline("text-classification", model=...)`
- Recordá importar `pandas` para crear el DataFrame
- Modelos sugeridos: `finiteautomata/beto-sentiment-analysis` o `pysentimiento/robertuito-sentiment-analysis`

---

### Parte 2: Extracción de información (40%)

1. Tomá 2 de las reseñas que creaste (las más largas y detalladas)
2. Cargá un modelo de NER en español
3. Extraé todas las entidades nombradas de esas reseñas
4. Organizá la información en categorías (personas, lugares, organizaciones)
5. Bonus: ¿Se mencionan nombres de platos? (Nota: el modelo podría no detectarlos como entidades, reflexioná sobre por qué)

**Pistas:**
- Usá `pipeline("ner", model=..., aggregation_strategy="simple")`
- Modelo sugerido: `mrm8488/bert-spanish-cased-finetuned-ner`
- Recordá iterar sobre los resultados para organizarlos por tipo

---

### Parte 3: Sistema de preguntas y respuestas (30%)

1. Escribí un texto con información del restaurante (menú, horarios, ubicación, políticas de reservas, precios promedio, etc.). Mínimo 4-5 oraciones.
2. Cargá un modelo de Question Answering en español
3. Formulá al menos 4 preguntas que un cliente podría hacer
4. Generá respuestas automáticas usando el modelo
5. Mostrá cada pregunta con su respuesta y el nivel de confianza del modelo

**Pistas:**
- Usá `pipeline("question-answering", model=...)`
- Modelo sugerido: `PlanTL-GOB-ES/roberta-base-bne-sqac`
- La función necesita dos parámetros: `question=` y `context=`

---

### Bonus (opcional): Integración completa

Si terminaste las tres partes, intentá crear una función que:
1. Reciba una reseña de cliente como input
2. Analice el sentimiento
3. Extraiga entidades mencionadas
4. Genere un resumen estructurado

Por ejemplo:
```
RESEÑA: "Fui ayer a la sucursal de Palermo y el mozo Juan me atendió bárbaro..."

ANÁLISIS:
- Sentimiento: POSITIVO (95% confianza)
- Empleado mencionado: Juan
- Sucursal: Palermo
- Recomendación: Enviar agradecimiento personalizado
```

---

### Criterios de evaluación

Evaluá tu propio trabajo considerando:

1. **Funcionalidad (50%):** ¿El código funciona sin errores? ¿Completaste las tres partes?
2. **Calidad de datos (20%):** ¿Las reseñas y preguntas son realistas? ¿El contexto tiene información útil?
3. **Presentación (20%):** ¿Los resultados se muestran de forma clara? ¿Usaste DataFrames o print statements organizados?
4. **Reflexión crítica (10%):** ¿Analizaste la calidad de las predicciones? ¿Identificaste limitaciones?

---

### Espacio para tu solución

Usá las celdas siguientes para resolver el desafío. Podés crear todas las celdas que necesites.


In [63]:
# Cargamos un modelo de análisis de sentimientos específico para español
clasificador = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Modelo de sentimientos cargado correctamente.")


# Creá una lista con al menos 5 reseñas ficticias
resenas = [
    "La parrilla del lugar es increoble, las carne en el punto justo y el servicio muy bueno tambien. Voy a traer a mis amigos a comer aca, a Juan le va a copar",
    "Pedí una milanesa y llegó fría, el pure parecia de paquete. Mi esposa Gimena casi me hace dormir afuera por llevarla ahi",
    "El ambiente es lindo, agradable pero las porciones muy chiquitas para lo que se paga.",
    "La atención del mozo Mario fue espectacular, todo lo que recomendo fue en 10. No se encuentra seguido atencion tan buena",
    "Todo estaba salado, el paro cardiaco me miro de cerca",
    "Aguante morirse de sed porque el mozo no venia nunca a molestarnos",
    "Mega divertido esperar afuera al rayo del sol por mas de dos horas"
]

print("Reseñas a analizar:")
for i, resena in enumerate(resenas, 1):
    print(f"{i}. {resena}")

# Analizamos las reseñas
resultados = clasificador(resenas)

# Creamos un DataFrame para visualizar mejor los resultados
df_resultados = pd.DataFrame({
    'Reseña': resenas,
    'Sentimiento': [r['label'] for r in resultados],
    'Confianza': [round(r['score'], 3) for r in resultados]
})

print("\nResultados del análisis de sentimientos:")
print(df_resultados)

# Identificamos cuántas reseñas son positivas, negativas y neutrales
positivas = df_resultados[df_resultados['Sentimiento'] == 'POS']
negativas = df_resultados[df_resultados['Sentimiento'] == 'NEG']
neutrales = df_resultados[df_resultados['Sentimiento'] == 'NEU']

print(f"\nResumen: {len(positivas)} positivas, {len(negativas)} negativas, {len(neutrales)} neutrales")


Device set to use cuda:0


Modelo de sentimientos cargado correctamente.
Reseñas a analizar:
1. La parrilla del lugar es increoble, las carne en el punto justo y el servicio muy bueno tambien. Voy a traer a mis amigos a comer aca, a Juan le va a copar
2. Pedí una milanesa y llegó fría, el pure parecia de paquete. Mi esposa Gimena casi me hace dormir afuera por llevarla ahi
3. El ambiente es lindo, agradable pero las porciones muy chiquitas para lo que se paga.
4. La atención del mozo Mario fue espectacular, todo lo que recomendo fue en 10. No se encuentra seguido atencion tan buena
5. Todo estaba salado, el paro cardiaco me miro de cerca
6. Aguante morirse de sed porque el mozo no venia nunca a molestarnos
7. Mega divertido esperar afuera al rayo del sol por mas de dos horas

Resultados del análisis de sentimientos:
                                              Reseña Sentimiento  Confianza
0  La parrilla del lugar es increoble, las carne ...         POS      0.999
1  Pedí una milanesa y llegó fría, el pure pare

In [73]:
# PARTE 2: EXTRACCIÓN DE INFORMACIÓN (NER)
# Escribí tu código acá

extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens de la misma entidad
)

print("Modelo NER cargado correctamente.")

resenas_ner = [
    "La parrilla del lugar es increoble, las carne en el punto justo y el servicio muy bueno tambien. Voy a traer a mis amigos a comer aca, a Juan le va a copar",
    "Pedí una milanesa y llegó fría, el pure parecia de paquete. Mi esposa Gimena casi me hace dormir afuera por llevarla ahi",
]

print("Reseñas para análisis NER:")
for i, resena in enumerate(resenas_ner, 1):
    print(f"{i}. {resena}")

# Procesamos el texto de las reseñas
entidades = extractor_ner(resenas_ner[0] + " " + resenas_ner[1])

# Creamos un DataFrame para visualizar mejor
df_entidades = pd.DataFrame(entidades)

# Mostramos solo las columnas relevantes
print("\nEntidades detectadas:")
df_entidades[['entity_group', 'word', 'score']].round(3)


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Modelo NER cargado correctamente.
Reseñas para análisis NER:
1. La parrilla del lugar es increoble, las carne en el punto justo y el servicio muy bueno tambien. Voy a traer a mis amigos a comer aca, a Juan le va a copar
2. Pedí una milanesa y llegó fría, el pure parecia de paquete. Mi esposa Gimena casi me hace dormir afuera por llevarla ahi

Entidades detectadas:


,entity_group,word,score
0,PER,Juan,0.995
1,PER,Gimena,0.802


In [62]:
# PARTE 3: QUESTION ANSWERING - CORREGIDO
# Escribí tu código acá

from transformers import pipeline

print("Cargando modelo de Question Answering...")

# Usamos el modelo que SÍ funciona para español
try:
    qa_modelo = pipeline(
        "question-answering",
        model="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
    )
    print("✅ Modelo QA cargado correctamente")
except Exception as e:
    print(f"❌ Error con el modelo QA: {e}")
    print("🔄 Cargando modelo alternativo...")
    qa_modelo = pipeline(
        "question-answering",
        model="dccuchile/bert-base-spanish-wwm-uncased"
    )
    print("✅ Modelo QA alternativo cargado")

# Escribí un texto con información del restaurante
info_restaurante = """
El restaurante 'El Buen Sabor' está en Palermo, Buenos Aires.
Abre de martes a domingo de 12:00 a 16:00 y de 20:00 a 00:00 horas.
El menú incluye parrilla, empanadas, pizzas y pastas.
Los precios son: entrada $12000, plato principal $15000, postre $8000.
Las reservas se hacen por teléfono al 11-1234-5678.
El estacionamiento es gratis para clientes.
"""

print("\nInformación del restaurante:")
print(info_restaurante)

# Formulá preguntas simples y directas
preguntas = [
    "¿Cuáles son los horarios?",
    "¿Qué comida sirven?",
    "¿Cuánto cuesta un plato principal?",
    "¿Cómo hago una reserva?",
    "¿Hay estacionamiento?"
]

# Generá respuestas automáticas
print("\n" + "="*60)
print("RESPUESTAS AUTOMÁTICAS DEL SISTEMA")
print("="*60)

for pregunta in preguntas:
    try:
        respuesta = qa_modelo(question=pregunta, context=info_restaurante)
        print(f"\n❓ PREGUNTA: {pregunta}")
        print(f"✅ RESPUESTA: {respuesta['answer']}")
        print(f"📊 CONFIANZA: {respuesta['score']*100:.1f}%")
        print("-" * 50)
    except Exception as e:
        print(f"\n❓ PREGUNTA: {pregunta}")
        print(f"❌ ERROR: {e}")
        print("-" * 50)

Cargando modelo de Question Answering...


Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


✅ Modelo QA cargado correctamente

Información del restaurante:

El restaurante 'El Buen Sabor' está en Palermo, Buenos Aires.
Abre de martes a domingo de 12:00 a 16:00 y de 20:00 a 00:00 horas.
El menú incluye parrilla, empanadas, pizzas y pastas.
Los precios son: entrada $12000, plato principal $15000, postre $8000.
Las reservas se hacen por teléfono al 11-1234-5678.
El estacionamiento es gratis para clientes.


RESPUESTAS AUTOMÁTICAS DEL SISTEMA

❓ PREGUNTA: ¿Cuáles son los horarios?
✅ RESPUESTA: 20:00 a 00:00 horas
📊 CONFIANZA: 6.0%
--------------------------------------------------

❓ PREGUNTA: ¿Qué comida sirven?
✅ RESPUESTA: parrilla, empanadas, pizzas y pastas
📊 CONFIANZA: 51.9%
--------------------------------------------------

❓ PREGUNTA: ¿Cuánto cuesta un plato principal?
✅ RESPUESTA: $12000
📊 CONFIANZA: 29.0%
--------------------------------------------------

❓ PREGUNTA: ¿Cómo hago una reserva?
✅ RESPUESTA: teléfono al 11-1234-5678
📊 CONFIANZA: 53.5%
---------------------

In [ ]:
# BONUS (OPCIONAL): INTEGRACIÓN COMPLETA
# Escribí tu código acá si querés hacer el bonus



### Reflexión personal

Después de completar el ejercicio, respondé estas preguntas:

1. **¿Qué fue lo más difícil del ejercicio?**
   - (Escribí tu respuesta acá)

2. **¿Encontraste alguna limitación en los modelos?**
   - (Escribí tu respuesta acá)

3. **¿Cómo podrías mejorar este sistema para un caso real?**
   - (Escribí tu respuesta acá)

4. **¿Qué otras aplicaciones se te ocurren para estas técnicas en el contexto argentino?**
   - (Escribí tu respuesta acá)

---

---

## Reflexiones Finales del Curso

Completaste los cuatro ejercicios de NLP con Transformers, incluyendo un desafío autónomo. Vamos a reflexionar sobre lo aprendido:

### Ejercicio 1 - Moderación de Comentarios
- **Aprendiste:** A clasificar texto usando modelos de análisis de sentimientos
- **Aplicación:** Moderación automática de redes sociales, priorización de tickets de soporte
- **Limitaciones:** Los modelos pueden tener problemas con ironía, sarcasmo o lenguaje muy coloquial

### Ejercicio 2 - Extracción de CVs
- **Aprendiste:** A extraer entidades nombradas (personas, organizaciones, lugares) de texto
- **Aplicación:** Automatización de RRHH, análisis de documentos, extracción de información estructurada
- **Limitaciones:** Algunos modelos pueden confundir entidades o no detectar nombres poco comunes

### Ejercicio 3 - Chatbot de Soporte
- **Aprendiste:** A usar Question Answering para responder preguntas basadas en un contexto
- **Aplicación:** Chatbots, asistentes virtuales, sistemas de FAQ automáticas
- **Limitaciones:** El modelo solo puede responder sobre información presente en el contexto

### Ejercicio 4 - Desafío Autónomo
- **Aprendiste:** A combinar múltiples técnicas de NLP para resolver un problema real completo
- **Aplicación:** Sistemas integrados de análisis de feedback, gestión de reputación online
- **Habilidad clave:** Autonomía para investigar, implementar y evaluar soluciones de NLP

### Próximos pasos sugeridos

1. **Explorá más modelos** en [Hugging Face Hub](https://huggingface.co/models)
2. **Combiná técnicas:** Por ejemplo, usá clasificación de sentimientos + QA para un chatbot más inteligente
3. **Experimentá con otros idiomas** o dialectos regionales
4. **Investigá fine-tuning:** Aprendé a ajustar modelos con tus propios datos
5. **Desarrollá un proyecto propio:** Elegí un problema real que te interese y aplicá estas técnicas

---

## Glosario Técnico

### Conceptos fundamentales

**Transformer**  
Arquitectura de red neuronal basada en mecanismos de atención que revolucionó el NLP en 2017. Permite procesar secuencias de texto completas simultáneamente en lugar de palabra por palabra.

**Pipeline**  
Interfaz de alto nivel en Hugging Face que encapsula todo el proceso de preprocesamiento, inferencia y postprocesamiento de un modelo. Facilita el uso de modelos preentrenados con pocas líneas de código.

**Modelo preentrenado**  
Modelo de machine learning que fue entrenado previamente con grandes cantidades de datos. Puede usarse directamente o ajustarse (fine-tuning) para tareas específicas.

**Tokenización**  
Proceso de dividir texto en unidades más pequeñas (tokens) que el modelo puede procesar. Puede ser a nivel de palabras, subpalabras o caracteres.

### Tareas de NLP

**Text Classification (Clasificación de texto)**  
Tarea de asignar una o más etiquetas a un texto. Incluye análisis de sentimientos, detección de spam, clasificación de temas, etc.

**Sentiment Analysis (Análisis de sentimientos)**  
Subtipo de clasificación que identifica la polaridad emocional de un texto (positivo, negativo, neutral). Se usa en redes sociales, reseñas de productos, atención al cliente.

**Named Entity Recognition - NER (Reconocimiento de entidades nombradas)**  
Tarea de identificar y clasificar nombres propios en texto: personas (PER), organizaciones (ORG), ubicaciones (LOC), fechas, cantidades monetarias, etc.

**Question Answering - QA (Respuesta a preguntas)**  
Tarea de responder preguntas en lenguaje natural basándose en un contexto dado. El modelo extrae la respuesta directamente del texto proporcionado.

**Text Generation (Generación de texto)**  
Tarea de crear texto nuevo de manera coherente a partir de un prompt inicial. Incluye completado de texto, escritura creativa, chatbots conversacionales.

**Summarization (Resumen automático)**  
Tarea de condensar un texto largo en una versión más corta manteniendo la información más importante.

**Translation (Traducción automática)**  
Tarea de traducir texto de un idioma a otro usando modelos de secuencia a secuencia.

### Componentes técnicos

**Embedding (Representación vectorial)**  
Representación numérica de palabras o tokens como vectores en un espacio multidimensional. Palabras con significados similares tienen embeddings cercanos.

**Attention (Atención)**  
Mecanismo que permite al modelo enfocarse en diferentes partes del input al procesar cada elemento. Es el componente clave de la arquitectura Transformer.

**Fine-tuning (Ajuste fino)**  
Proceso de tomar un modelo preentrenado y entrenarlo adicionalmente con datos específicos de tu dominio para mejorar su desempeño en tu tarea particular.

**Inference (Inferencia)**  
Proceso de usar un modelo ya entrenado para hacer predicciones sobre datos nuevos. No implica entrenamiento, solo aplicación del modelo.

**Score / Confidence (Puntuación / Confianza)**  
Valor numérico (generalmente entre 0 y 1) que indica qué tan seguro está el modelo de su predicción. Valores más altos indican mayor confianza.

### Modelos mencionados

**BETO**  
Versión de BERT (Bidirectional Encoder Representations from Transformers) entrenada específicamente con texto en español. Usado en análisis de sentimientos y otras tareas de clasificación.

**RoBERTa**  
Variante optimizada de BERT con mejoras en el proceso de preentrenamiento. Usado para múltiples tareas de NLP en español.

**BERT (Bidirectional Encoder Representations from Transformers)**  
Modelo Transformer que lee texto bidireccionalmente (izquierda a derecha y derecha a izquierda simultáneamente) para comprender mejor el contexto.

**GPT (Generative Pre-trained Transformer)**  
Familia de modelos diseñados específicamente para generación de texto. Leen texto de izquierda a derecha y predicen la siguiente palabra.

### Plataformas y librerías

**Hugging Face**  
Plataforma y empresa que desarrolla herramientas de NLP de código abierto. Su librería Transformers es el estándar de facto para trabajar con modelos de lenguaje.

**Hugging Face Hub**  
Repositorio online con miles de modelos preentrenados, datasets y demos interactivas. Permite compartir y descargar modelos fácilmente.

**PyTorch / TensorFlow**  
Frameworks de deep learning usados como backend por la librería Transformers. PyTorch es más común en investigación, TensorFlow en producción.

### Métricas y evaluación

**Label (Etiqueta)**  
Categoría asignada por el modelo a un texto. En análisis de sentimientos: POS (positivo), NEG (negativo), NEU (neutral).

**Aggregation strategy (Estrategia de agregación)**  
En NER, método para combinar tokens que pertenecen a la misma entidad. Por ejemplo, "Buenos" y "Aires" se agrupan en una sola entidad "Buenos Aires".

**Context (Contexto)**  
En QA, el documento o párrafo que contiene la información necesaria para responder la pregunta. El modelo busca la respuesta dentro de este contexto.

---

## Recursos adicionales

- **Documentación oficial de Transformers:** https://huggingface.co/docs/transformers
- **Modelos en español:** https://huggingface.co/models?language=es
- **Curso gratuito de Hugging Face:** https://huggingface.co/course
- **Comunidad en español:** https://huggingface.co/spaces

---

*Este cuaderno fue diseñado con fines educativos para estudiantes de NLP en Argentina.*